In [1]:
from __future__ import division
from MarkovModel import ThirdOrderMarkovModel
import pandas as pd
import numpy as np

import json

from  Preprocesscopy import Preprocesscopy

from sklearn.cross_validation import train_test_split

In [2]:
file_path = '/home/rui/Documents/MLProject/data/modified-crab/trips-in-sanfran.csv'
city = 'san_francisco'
tile_size = 0.360
la_size = 0.003234
long_size = 0.004049
start_id = 0
freq_threshold = 0

In [3]:
data = Preprocesscopy(file_path, city, tile_size, la_size, long_size, freq_threshold)

In [4]:
data.read_dataset()
print "done read dataset"
data.split_train_test()
print "done split dataset"
data.find_significant_region(start_id)
print "done find significant region"
data.filter_low_frequency_trajectory()
print "done filter low frequency traj"
data.refind_significant_region(start_id)
print "done refind significant region"
data.convert_train_traj_to_region_id()
print "done convert train traj to region id"
data.convert_test_traj_to_region_id()
print "done convert test traj to region id"

done read dataset
done split dataset
done find significant region
done filter low frequency traj
done refind significant region
done convert train traj to region id
done convert test traj to region id


In [5]:
taxi_train = data.taxi_train
taxi_test = data.taxi_test

In [6]:
# define a function to convert trajactro to a numpy array
def trace2train_format(training_data):
    train_locations = []
    train_lengths = []
    for row_index, row in training_data.iterrows():
        single_trip = row[3]
        trip_length = len(single_trip)
        
        train_lengths.append(trip_length)
        
        for i in xrange(trip_length):
            train_locations.append(single_trip[i])

    train_locations = np.array(train_locations)
    train_lengths = np.array(train_lengths)
    return train_locations, train_lengths

In [7]:
num_states = len(data.region_dict)
train_locations, train_lengths = trace2train_format(taxi_train)

thirdmm = ThirdOrderMarkovModel(num_states)
thirdmm.fit(train_locations, train_lengths)

# test

In [8]:
not_predicted = []
total_prediction = 0
correct_prediction = 0
for row_index, row in taxi_test.iterrows():
    single_trip = row[3]
    trip_length = len(single_trip)
    if trip_length >= 4:
        for i in xrange(trip_length - 3):
            total_prediction += 1
            next_prediction = thirdmm.predict([single_trip[i], single_trip[i+1], single_trip[i+2]])
            if next_prediction == -1:
                not_predicted.append(1)
            if next_prediction == single_trip[i+2]:
                correct_prediction += 1
    else:
        continue
accuracy = correct_prediction / total_prediction
    
print "The accuracy is %f " %(accuracy)

The accuracy is 0.220545 
